In [ ]:
import pandas as pd
import requests
import os
from Bio import SeqIO as seqio
import zipfile
import numpy as np
from kinaid.session import Session
from kinaid.matching import MatchWithMapping, Scoring, PeptideBackground, PWM_Matrices
from kinaid.utility import Utility
from kinaid.ortholog import OrthologManager

def download_file(url : str,
                filename : str):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }
    with requests.get(url, headers=headers, stream=True) as response:
        response.raise_for_status()
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192): 
                if chunk: 
                    file.write(chunk)

def get_protein_sequence(seq, phospho_position):

    final_seq = None
    if phospho_position < 5 :
        final_seq =  '_'*(5 - phospho_position) + seq[0:phospho_position+6]
    elif phospho_position > len(seq) - 6:
        final_seq =  seq[phospho_position-5:] + '_'*(phospho_position - len(seq) + 6)  
    else:
        final_seq = seq[phospho_position-5:phospho_position+6]
    
    if final_seq[5] !='S' and final_seq[5] != 'T' and final_seq[5] != 'Y':
        print(final_seq)
        return None
    return final_seq

data_dir = './data'
test_dir = './test'
proteomes_dir = os.path.join(test_dir, 'proteomes')

if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    
johnson_ST_matrices_file = os.path.join(data_dir,'ST-Kinases.xlsx')
johnson_Y_matrices_file = os.path.join(data_dir,'Y-Kinases.xlsx')
densitometry_file = os.path.join(data_dir,'ST-Kinases_densitometry.xlsx')

ST_matrices = PWM_Matrices(johnson_ST_matrices_file)
ST_matrices.add_densitometry(densitometry_file)

Y_matrices = PWM_Matrices(johnson_Y_matrices_file)

st_scoring = Scoring(ST_matrices)
y_scoring = Scoring(Y_matrices)

ochoa_background_file = os.path.join(data_dir, 'johnson_ochoa_background_wfav.tsv')

tyrosine_background_file = os.path.join(data_dir, 'johnson_tyrosine_background.tsv')

ST_Background = PeptideBackground(ochoa_background_file)
Y_Background = PeptideBackground(tyrosine_background_file)

human_kinase_file = os.path.join(data_dir,'human_kinases_final.tsv')

ortholog_manager = OrthologManager('orthologs', human_kinase_file=human_kinase_file)

background = {'ST':ST_Background, 'Y':Y_Background}
scoring = {'ST':st_scoring, 'Y':y_scoring}

# Yeast Experiments

### Leutert, M., Barente, A.S., Fukuda, N.K. et al. The regulatory landscape of the yeast phosphoproteome. Nat Struct Mol Biol 30, 1761–1773 (2023).


In [2]:
def clean_phosphopeptides(phosphorylation_file, fasta_file, output_file) :
    exp_df = pd.read_csv(phosphorylation_file)

    #read in the fasta file
    fasta_dict = seqio.to_dict(seqio.parse(fasta_file, 'fasta'))

    exp_df = exp_df[exp_df['p_residue'].isin(['S', 'T', 'Y'])]

    systematic_name_to_SGD_dict = {name:record.description.split(', ')[0].split(' ')[2] for name,record in fasta_dict.items()}
    #systematic_name_to_SGD_dict = {name:SGD.split(':')[1] for name,SGD in systematic_name_to_SGD_dict.items()}

    exp_df['SGDID'] = exp_df['systematic_name'].map(systematic_name_to_SGD_dict)

    systematic_name_to_seq_dict = {name:str(record.seq) for name,record in fasta_dict.items()}
    exp_df['sequence'] = (exp_df['systematic_name'].map(systematic_name_to_seq_dict))

    #remove rows with missing sequences
    exp_df = exp_df[~exp_df['sequence'].isna()]

    #remove rows with stop codons
    exp_df['sequence'] = exp_df['sequence'].str.rstrip('*')

    #get sequence lengths
    exp_df['seq_len'] = exp_df['sequence'].str.len()

    #keep only rows where the sequence length is greater than or equal to the position of the phosphorylation site
    exp_df = exp_df[exp_df['seq_len'] >= exp_df['p_position']]

    exp_df['sanity_check'] = exp_df.apply(lambda row: row['sequence'][row['p_position']-1] == row['p_residue'], axis=1)
    
    exp_df = exp_df[exp_df['sanity_check']]

    #if p_position is less than 6, add 5 - p_position number of _'s to the beginning of the sequence
    exp_df['sequence'] = exp_df.apply(lambda row: '_'*(6 - row['p_position']) + row['sequence'], axis=1)

    #if p_position is greater than seq_len - 6, add p_position - (seq_len - 5) number of _'s to the end of the sequence
    exp_df['sequence'] = exp_df.apply(lambda row: row['sequence'] + '_'*(row['p_position'] - (row['seq_len'] - 5)), axis=1)

    #make window around p_position if p_position is greater than 5 from p_position - 6 to p_position + 5, else from 0 to 10
    exp_df['window'] = exp_df.apply(lambda row: row['sequence'][row['p_position']-6:row['p_position']+5] if row['p_position'] > 5 else row['sequence'][0:11], axis=1)

    exp_df['adj_p_value'] = exp_df['adj_p_value'].apply(lambda x: -1*np.log10(x))

    exp_df_filtered = exp_df[['SGDID', 'p_position', 'window', 'fc_log2', 'adj_p_value']].copy()

    exp_df_filtered.rename(columns={'SGDID':'SGD', 'p_position':'site', 'window':'peptide', 'fc_log2':'log2fc', 'adj_p_value':'adjpvalue'}, inplace=True)

    exp_df_filtered.to_csv(output_file, index=False)

In [19]:
published_yeast_file = 'yeast-perturbation.xlsx'
published_yeast_path = os.path.join(test_dir, published_yeast_file)


import gzip
yeast_proteome_path = os.path.join(proteomes_dir, 'yeast_proteome.fasta')

if not os.path.exists(proteomes_dir):
    os.makedirs(proteomes_dir)
    
if not os.path.exists(yeast_proteome_path):
    print('Downloading yeast proteome')
    Utility.download_file('http://sgd-archive.yeastgenome.org/sequence/S288C_reference/orf_protein/orf_trans.fasta.gz', 'temp.gz')
    with gzip.open('temp.gz', 'rb') as f_in:
        with open(yeast_proteome_path, 'wb') as f_out:
            f_out.write(f_in.read())
    os.remove('temp.gz')
    

if not os.path.exists(published_yeast_path):
    print('Downloading yeast perturbation data')
    Utility.download_file('https://static-content.springer.com/esm/art%3A10.1038%2Fs41594-023-01115-3/MediaObjects/41594_2023_1115_MOESM9_ESM.xlsx',
                  published_yeast_path)


HOG1_experiment_file = os.path.join(test_dir,'HOG1_exp.csv')

if not os.path.exists(HOG1_experiment_file):
    df = pd.read_excel(published_yeast_path, sheet_name='p_site_diff_reg')

HOG1_phosphopeptides_file = 'yeast_HOG1_KC_phosphopeptides.csv'
HOG1_path = os.path.join(test_dir, HOG1_phosphopeptides_file)
if not os.path.exists(HOG1_path):
    df[df['treatment_id'] == 'KC'].to_csv(HOG1_path, index=False)
    clean_phosphopeptides(HOG1_path, yeast_proteome_path, HOG1_experiment_file)
    
column_names_dict={'id':'SGD', 'site':'site', 'peptide':'peptide', 'log2fc':'log2fc', 'dependent':'adjpvalue'}

HOG1_df = pd.read_csv(HOG1_experiment_file)
print(len(HOG1_df))
session = Session(0, 'yeast', HOG1_df, column_names_dict, scoring, background, ortholog_manager, id_type='SGD', ambiguous=True, debug=True)


5258
Elapsed time for percentiles and matches: 6.01 seconds


In [ ]:
SNF1_experiment_file = os.path.join(test_dir,'SNF1_exp.csv')

if not os.path.exists(SNF1_experiment_file):
    df = pd.read_excel(published_yeast_path, sheet_name='p_site_diff_reg')

SNF1_phosphopeptides_file = 'yeast_SNF1_GL_phosphopeptides.csv'
SNF1_path = os.path.join(test_dir, SNF1_phosphopeptides_file)
if not os.path.exists(SNF1_path):
    df[df['treatment_id'] == 'GL'].to_csv(SNF1_path, index=False)
    clean_phosphopeptides(SNF1_path, yeast_proteome_path, SNF1_experiment_file)


# Mouse Experiment
### Huttlin EL, Jedrychowski MP, Elias JE, Goswami T, Rad R, Beausoleil SA, Villén J, Haas W, Sowa ME, Gygi SP. A tissue-specific atlas of mouse protein phosphorylation and expression. Cell. 2010 Dec 23;143(7):1174-89. 

In [4]:
def region_check(region) :
    regions = [int(r) for r in region.split(';')]
    if regions[0] != regions[1] :
        return False
    else :
        return True

def extract_MGI_id(gene_ids) :
    if type(gene_ids) != str :
        return None
    if(len(gene_ids) == 0) :
        return None
    ids = gene_ids.split('; ')
    
    for id in ids :
        if 'MGI' in id :
            #expected format MG:MGI:1234567
            mgi = id.split(':')[2]
            return mgi
    else :
        return None
    
#function to process peptides
def extract_peptide(opeptide) :
    #split peptide by *
    peptide = opeptide.split('*')

    #if there are not 2 parts, return None
    if len(peptide) != 2 :
        print(opeptide)
        return None
    #get last 5 elements from first part
    first_part = peptide[0][-6:]

    #get first 5 elements from second part
    second_part = peptide[1][:5]

    #combine
    return first_part + second_part

In [5]:
published_mouse_data = 'mouse-phosphoproteome.zip'
mouse_phosphopeptides_file = os.path.join(test_dir,'cell5459mmc2.xlsb')

if not os.path.exists(mouse_phosphopeptides_file):
    print('Downloading mouse phosphoproteome data')
    download_file('https://ars.els-cdn.com/content/image/1-s2.0-S0092867410013796-mmc2.zip',
                  published_mouse_data)
    #extract the zip file into the current directory
    with zipfile.ZipFile(published_mouse_data, 'r') as zip_ref:
        zip_ref.extractall(test_dir)
        #delete the zip file
        os.remove(published_mouse_data)

df = pd.read_excel(mouse_phosphopeptides_file, engine='pyxlsb', sheet_name='phosphoMouse_siteList_withAllPr')
df = df[["Gene ID's", 'entropy', 'motif peptide', 'Region']].copy()

#rename columns to id, log2fc, peptide, site
df.columns = ['id', 'log2fc', 'peptide', 'site']

display(df.head())


,id,log2fc,peptide,site
0,EG:72925; EN:ENSMUSG00000036469; MG:MGI:192017...,1.611158,SNISKAS*SPTTGT,52;52
1,EG:72925; EN:ENSMUSG00000036469; MG:MGI:192017...,1.611158,NISKASS*PTTGTA,53;53
2,EG:57438; EN:ENSMUSG00000026977; MG:MGI:193105...,1.611158,MEDLETS*EDEF--,689;689
3,EG:71779; EN:ENSMUSG00000025702; MG:MGI:191902...,1.389903,SAFSRTS*VTPSNQ,66;67
4,EG:71779; EN:ENSMUSG00000025702; MG:MGI:191902...,1.389903,RTSVTPS*NQDICR,71;71


In [6]:
df['region_check'] = df['site'].apply(region_check)
df['site'] = df['site'].apply(lambda x: x.split(';')[0])

#remove rows where region is not the same
df = df[df['region_check']].copy()

df['id'] = df['id'].apply(extract_MGI_id)

#remove rows where id is None
df = df[df['id'].notna()].copy()

df['peptide'] = df['peptide'].apply(extract_peptide)

#remove rows where peptide is None
df = df[df['peptide'].notna()].copy()

#remove duplicate rows
df = df.drop_duplicates().copy()

#how many rows are left?
print(len(df))

#write to csv
mouse_experiment_file = os.path.join(test_dir,'mouse_complete_phosphopeptides.csv')
df.to_csv(mouse_experiment_file, index=False)
display(df)

QQICRDSEDEEAW
29978


,id,log2fc,peptide,site,region_check
0,1920175,1.611158,NISKASSPTTG,52,True
1,1920175,1.611158,ISKASSPTTGT,53,True
2,1931053,1.611158,EDLETSEDEF-,689,True
4,1919029,1.389903,TSVTPSNQDIC,71,True
5,1858916,0.982129,FPECDSDEDEE,206,True
...,...,...,...,...,...
35959,2444286,1.664698,RLRLLSFRSME,1535,True
35960,2444286,1.757968,LLSFRSMEETR,1538,True
35961,2444286,1.576964,DRPVSSPGEAE,2407,True
35962,1920453,1.590218,RDSWVSPRKRR,89,True


In [7]:

column_names_dict={'id':'id', 'site':'site', 'peptide':'peptide', 'log2fc':'log2fc', 'dependent':None}

session = Session(0, 'mouse', df, column_names_dict, scoring, background, ortholog_manager, id_type='MGI', ambiguous=True, debug=True)


634      2442238_223
964      1351658_194
1225     1890410_263
1226     1890410_267
            ...     
26834      1914395_8
26921     107749_136
26938     1913564_96
26956      98809_283
27900     894652_359
Name: __ENTRY_ID, Length: 102, dtype: object
Elapsed time for percentiles and matches: 92.41 seconds


# Fly Experiment

### Yang L & Zhu AW et al. ERK controls embryonic cleavage divisions in Drosophila. Developmental Cell (2024)

In [20]:
published_fly_data = os.path.join(test_dir,'optosos_fly_clean.csv')
df = pd.read_csv(published_fly_data)
print(len(df))
display(df.head())

4139


,Motif,log2FC,Uniprot,Site Position,neglogadjp,adjp
0,KWRDESDDEE,-0.000618,Q9VH95,116,0.003350,0.992315
1,EGGKESPPPP,0.001109,Q9VGU5,589,0.003619,0.991701
2,GKNDDSDEEE,-0.000843,Q9VZP5,76,0.003619,0.991701
3,KLRKVSLDEP,-0.001183,A1Z898,424,0.003804,0.991280
4,LHDEASDDKS,0.003086,Q8IMV6,205,0.006157,0.985923


In [9]:
column_names_dict={'id':'Uniprot', 'site':'Site Position', 'peptide':'Motif', 'log2fc':'log2FC', 'dependent': 'neglogadjp'}
session = Session(0, 'fly', df, column_names_dict, scoring, background, ortholog_manager, id_type='UniProtKB', ambiguous=True, debug=True)



Elapsed time for percentiles and matches: 7.98 seconds


# Zebrafish experiment

### Kwon, O.K., Kim, S.J., Lee, Y.-M., Lee, Y.-H., Bae, Y.-S., Kim, J.Y., Peng, X., Cheng, Z., Zhao, Y. and Lee, S. (2016), Global analysis of phosphoproteome dynamics in embryonic development of zebrafish (Danio rerio). Proteomics, 16: 136-149

In [10]:
published_zebrafish_data = os.path.join(test_dir,'Kwon_zebrafish.xls')
final_zebrafish_data = os.path.join(test_dir,'zebrafish_phosphopeptides.csv')

proteome_url = 'https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000000437/UP000000437_7955.fasta.gz'

if not os.path.exists(final_zebrafish_data):
    print('Downloading zebrafish phosphoproteome data')
    #download_file('https://analyticalsciencejournals.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fpmic.201500017&file=pmic12168-sup-0003-tableS3.xls',
    #              'Kwon_zebrafish.xls')
    df = pd.read_excel(published_zebrafish_data, sheet_name='phosphorylation site', skiprows=2)
    df.to_csv(final_zebrafish_data, index=False)

proteome_fasta = os.path.join(proteomes_dir,'zebrafish_proteome.fasta')

if not os.path.exists(proteome_fasta):
    print('Downloading zebrafish proteome')
    download_file(proteome_url, 'zebrafish_proteome.fasta.gz')
    os.system('gunzip zebrafish_proteome.fasta.gz')
    os.rename('zebrafish_proteome.fasta', proteome_fasta)

In [11]:
df = pd.read_csv(final_zebrafish_data)
display(df.head())

zebrafish_fasta_dict = seqio.to_dict(seqio.parse(proteome_fasta, 'fasta'), key_function=lambda x: x.id.split('|')[1])
print(f'Number of proteins in zebrafish proteome: {len(zebrafish_fasta_dict)}')

df = df[df['Phospo Amino'].isin(['S', 'T', 'Y'])]
df['exists_in_proteome'] = df['UniProtKB ID'].apply(lambda x: x in zebrafish_fasta_dict)
df = df[df['exists_in_proteome']]

print(f'Number of proteins in zebrafish proteome that have phosphosites: {len(df)}')


,Tage,UniProtKB ID,Phospho Position,Phospo Amino,Motif Logo
0,A0JMB8-889,A0JMB8,889,S,******S**D***
1,A0MQ61-1201,A0MQ61,1201,S,******S**D***
2,A1L1W1-687,A1L1W1,687,S,******S**D***
3,A1L260-299,A1L260,299,S,******S**D***
4,A2RRV2-60,A2RRV2,60,S,******S**D***


Number of proteins in zebrafish proteome: 25999
Number of proteins in zebrafish proteome that have phosphosites: 1078


In [12]:
df['peptide'] = df.apply(lambda x: get_protein_sequence(str(zebrafish_fasta_dict[x['UniProtKB ID']].seq), x['Phospho Position'] - 1 ), axis=1)
df = df.rename(columns={'Phospho Position': 'site'})

df = df[['peptide', 'UniProtKB ID', 'site',]]
#drop rows with missing peptides
df = df.dropna(subset=['peptide'])

zebrafish_experiment_file = os.path.join(test_dir,'zebrafish_experiment.csv')
df.to_csv(zebrafish_experiment_file, index=False)

display(df)

GQPGFKFTVAE
RSRGEAEDSSS
HKSKSEEVGTV
TSCETKKESLR
EAPATRRSSRR
PAITSDPEQDE
ARRLLGADNAA
KSRGKLRRKVH
FKNEQEVVSDD
HQRRALTPSAS
VLPLQPLIEKP
PVTRPHTTKRG
LESGSKVTEAC
NADIPNEPESP
GGEYRPKVYDQ
NSVSSDLKGSP
QLSPSRENETD
TPAVKNLDMSS
KKNQEKSLSDR
PGPDGPMGGMG
PNNMAGMNNPP
VQQQLQQKQQQ
VDLIPEPDTTE
AEEEEVVTKNV
ISKMAAISKAG
AAFAEKNRGRV
RGRVMGGLPDV
WSPSHQRRALT
KSLSDRSKANK
TSRKPEHRDDT
PQSPSKSTSSN
AKASKAPVSPH
TEMQRHYVMYY
KAAAFAEKNRG
RIKDEFQFLQA
PKIRQRIPFDL
WSWAVQWLQKK
LRPVGDSSVGR
APHQPGQPGFK
VPVDAPNPLTS
MRVLLHHQQTD
KATAEQDNIQV
IRSLFVCRHIK
ALQEALKEKQQ
PEAVKRKSSTD
KDAAKEASASF
SVKKGERELKI
GERELKINDRV
EGLPNHVHQGQ
RLRQQLKVPDK
VIRSLFVCRHI
RPPTPPPVEEV
SRRSRGEAEDS


,peptide,UniProtKB ID,site
0,PRKADSGKDLL,A0JMB8,889
3,PKGRKSSPDVT,A1L260,299
6,RGLSASNVDLT,A4IG65,606
7,TEKEDSDSDDV,A5PLK1,177
10,IPLNKSHNDFV,B3DLH3,106
...,...,...,...
3488,ETKRISVTQIP,Q8JG61,830
3489,LLDAESTRGDD,Q8UVJ6,59
3490,_MKGDTPVNST,Q90Z40,5
3496,LGDDESSSVST,Q9W6A5,333


In [13]:
df = pd.read_csv(zebrafish_experiment_file)
column_names_dict={'id':'UniProtKB ID', 'site':'site', 'peptide':'peptide', 'log2fc':None, 'dependent': None}

session = Session(0, 'zebrafish', df, column_names_dict, scoring, background, ortholog_manager, id_type='UniProtKB', ambiguous=True, debug=True)


Elapsed time for percentiles and matches: 3.32 seconds


# Worm experiment

### Li WJ, Wang CW, Tao L, Yan YH, Zhang MJ, Liu ZX, Li YX, Zhao HQ, Li XM, He XD, Xue Y, Dong MQ. Insulin signaling regulates longevity through protein phosphorylation in Caenorhabditis elegans. Nat Commun. 2021 Jul 27;12(1):4568.

In [14]:
published_worm_data_url = 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8316574/bin/41467_2021_24816_MOESM3_ESM.xlsx'
published_worm_data = os.path.join(test_dir,'worm_phosphosites.xlsx')

if not os.path.exists(published_worm_data):
    print('Downloading worm phosphoproteome data')
    download_file(published_worm_data_url, published_worm_data)

df = pd.read_excel(published_worm_data, sheet_name='Identified phosphosites', skiprows=3)
display(df.head())

#proteome_url = 'https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/reference_proteomes/Eukaryota/UP000001940/UP000001940_6239.fasta.gz'
proteome_url = 'https://rest.uniprot.org/uniprotkb/stream?format=fasta&query=%28proteome%3AUP000001940%29'
proteome_fasta = os.path.join(test_dir,'worm_proteome.fasta')

if not os.path.exists(proteome_fasta):
    print('Downloading worm proteome')
    download_file(proteome_url, proteome_fasta)
    #os.system('gunzip worm_proteome.fasta.gz')

df = df[df['Residue'].isin(['S', 'T', 'Y'])]

df = df[['UniProt ID', 'Position', 'WT_Bio1', 'WT_Bio2', 'WT_Bio3', 'daf-2_Bio1', 'daf-2_Bio2', 'daf-2_Bio3', 'daf-2_Bio4']]
df = df.groupby(['UniProt ID', 'Position']).max().reset_index()

#create a column WT_Bio that is True if at least two of the WT_Bio1, WT_Bio2, WT_Bio3 columns have a value greater than 0
df['WT_Bio_exist'] = ((df['WT_Bio1'] > 0).astype(int) + (df['WT_Bio2'] > 0).astype(int) + (df['WT_Bio3'] > 0).astype(int)) > 1
df['daf-2_Bio_exist'] = ((df['daf-2_Bio1'] > 0).astype(int) + (df['daf-2_Bio2'] > 0).astype(int) + (df['daf-2_Bio3'] > 0).astype(int) + (df['daf-2_Bio4'] > 0).astype(int)) > 1

#average the non-zero values of WT_Bio1, WT_Bio2, and WT_Bio3 and save result in a new column called WT_Bio
df = df[(df["WT_Bio_exist"]) & (df["daf-2_Bio_exist"])]
df['WT_Bio'] = df.apply(lambda x: [x['WT_Bio1'], x['WT_Bio2'], x['WT_Bio3']], axis=1)
df['WT_Bio'] = df['WT_Bio'].apply(lambda x: np.mean([i for i in x if i > 0]))

#average the non-zero values of daf-2_Bio1, daf-2_Bio2, daf-2_Bio3, and daf-2_Bio4 and save result in a new column called daf-2_Bio
df['daf-2_Bio'] = df.apply(lambda x: [x['daf-2_Bio1'], x['daf-2_Bio2'], x['daf-2_Bio3'], x['daf-2_Bio4']], axis=1)
df['daf-2_Bio'] = df['daf-2_Bio'].apply(lambda x: np.mean([i for i in x if i > 0]))

#calculate the log2 fold change of daf-2_Bio over WT_Bio and save result in a new column called log2_fold_change
df['log2fc'] = np.log2(df['daf-2_Bio'] / df['WT_Bio'])

df = df[['UniProt ID','Position','log2fc']]

display(df.head(10))

worm_fasta_dict = seqio.to_dict(seqio.parse(proteome_fasta, 'fasta'), key_function=lambda x: x.id.split('|')[1])
print(f'Number of proteins in worm proteome: {len(worm_fasta_dict)}')

df['exists_in_proteome'] = df['UniProt ID'].apply(lambda x: x in worm_fasta_dict)
#print the phosphosites that are not in the proteome
print(df[~df['exists_in_proteome']])

#keep only the phosphosites that are in the proteome
df = df[df['exists_in_proteome']]

df.drop(columns=['exists_in_proteome'], inplace=True)

df['is_valid'] = df.apply(lambda x: x['Position'] < len(worm_fasta_dict[x['UniProt ID']].seq), axis=1)

#keep only the phosphosites that are valid
df = df[df['is_valid']]
print(f'Number of valid phosphosites: {len(df)}')

df.drop(columns=['is_valid'], inplace=True)

df['peptide'] = df.apply(lambda x: get_protein_sequence(worm_fasta_dict[x['UniProt ID']].seq, x['Position'] - 1), axis=1)

df = df.dropna(subset=['peptide'])


worm_experiment_file = os.path.join(test_dir,'worm_experiment.csv')
df.to_csv(worm_experiment_file, index=False)


,Phosphopeptide (phosphorylation marked as * at the right side of S/T/Y residues),Charge state,UniProt ID,Position,Residue,Protein,Gene,WT_Bio1,WT_Bio2,WT_Bio3,...,daf-2_Bio3,daf-2_Bio4,daf-16; daf-2_Bio1,daf-16; daf-2_Bio2,daf-16; daf-2_Bio3,daf-16; daf-2_Bio4,daf-16_Bio1,daf-16_Bio2,daf-16_Bio3,daf-16_Bio4
0,APGNQAQVRKLS*QVMQHGR,2,A4F337,75,S,"Protein 2RSSE.1, isoform a",2RSSE.1,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,AAAFASNPS*HSLDYQEVGASNPR,3,U4PAK0,427,S,"Protein 2RSSE.1, isoform b",2RSSE.1,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.689184,0.000000,0.000000,0.0
2,AAAFASNPSHS*LDYQEVGASNPR,3,U4PAK0,429,S,"Protein 2RSSE.1, isoform b",2RSSE.1,0.000000,5.320352,8.408425,...,1.946737,6.297725,10.161053,0.0,12.672665,18.279308,10.756736,11.226915,11.707134,0.0
3,AAAFASNPSHS*LDYQEVGASNPR,2,U4PAK0,429,S,"Protein 2RSSE.1, isoform b",2RSSE.1,0.000000,0.000000,0.000000,...,0.000000,0.000000,3.387018,0.0,0.000000,0.000000,0.000000,0.000000,14.048561,0.0
4,GHTS*VEDDTWLAEVVPHDEIPK,3,U4PAK0,447,S,"Protein 2RSSE.1, isoform b",2RSSE.1,1.455456,7.980528,5.605617,...,17.520635,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


,UniProt ID,Position,log2fc
3,A0A061ACM2,442,-0.917498
7,A0A061ACS9,987,-0.689919
8,A0A061ACS9,995,-0.087314
10,A0A061ACS9,1024,0.232822
11,A0A061ACS9,1028,-0.111178
50,A0PD34,134,-1.549724
53,A3FPJ7,34,0.395414
55,A3KFD2,345,-0.330772
57,A3KFD2,354,-0.681597
64,A3QM97,683,0.032887


Number of proteins in worm proteome: 26703
      UniProt ID  Position    log2fc  exists_in_proteome
122       A7DT28       556 -0.775533               False
123       A7DT28       686 -0.366023               False
125       A7DT28       807 -1.661523               False
129       A7DT28      1355 -0.292911               False
130       A7DT28      1357 -0.292911               False
...          ...       ...       ...                 ...
15070     Q9XVY4       749 -0.131792               False
15174     Q9XWG7       377  0.692752               False
15374   Q9Y1J3-2       255 -0.039455               False
15377     S6EZL6        11  0.143245               False
15428     V6CJB8       573 -0.282021               False

[261 rows x 4 columns]
Number of valid phosphosites: 4184
ETGGYITLGYR
SRSGIDVRRND
SSRVTRDEQTV
EEEVEADYAPS
AFSQSPHRDWS
QSPHRDWSAST
WSASTVYDRVY
ASTVYDRVYNS
STVYDRVYNSN
STQITETITRT
QITETITRTTH
SVRTQEFAERI
PIQGMDTPFDH
EKRYFGDTLEK
DKKPNVPKKRA
AGPPPPVIEIS
KDGKEGKPFDF
IKLGKEDQLR

In [15]:
df = pd.read_csv(worm_experiment_file)
print(len(df))
column_names_dict={'id':'UniProt ID', 'site':'Position', 'peptide':'peptide', 'log2fc':'log2fc', 'dependent': None}

session = Session(0, 'worm', df, column_names_dict, scoring, background, ortholog_manager, id_type='UniProtKB', ambiguous=True, debug=True)


4061
Elapsed time for percentiles and matches: 6.28 seconds


# Human experiment
### Humphrey SJ, Yang G, Yang P, Fazakerley DJ, Stöckli J, Yang JY, James DE. Dynamic adipocyte phosphoproteome reveals that Akt directly regulates mTORC2. Cell Metab. 2013 Jun 4;17(6):1009-1020.

In [16]:
published_human_url = 'https://ars.els-cdn.com/content/image/1-s2.0-S1550413113001526-mmc2.xlsx'
published_human_data = os.path.join(test_dir,'human_phosphosites.xlsx')

if not os.path.exists(published_human_data):
    print('Downloading human phosphoproteome data')
    download_file(published_human_url, published_human_data)

df = pd.read_excel(published_human_data, sheet_name='Inhibitor screen regulated')
display(df.head())



,Gene Name,Phosphorylated Amino Acid,IPI 3.68 Position in Protein,IPI,Site Localization Class,Localization Probability,Sequence Window,Inhibitor Exps Insulin/Starved Median (log2),LY Screen Insulin Regulated Direction,LY Exps Insulin/Starved Median (log2),...,Andromeda Search Score,Uniprot ID,Proteins IPI IDs,Positions,Leading Proteins,Protein Descriptions,Modified Sequence,Gene Names,Protein Names,PEP
0,39326,T,426,IPI00224626,Class I,0.999318,QQNSSRTLEKNKK,-1.41,Negative,-1.46,...,261.67,Q3UGE0;Q3UNN1;Q5DTS3;Q8C2A3;O55131,IPI00224626;IPI00874440,426,IPI00224626,cell division cycle 10 homolog;Septin-7,_IIEQQNSSRT(ph)IEK_,Sept7;mKIAA4020;Cdc10;Sept7,Putative uncharacterized protein;MKIAA4020 pro...,2.730000e-55
1,1600027N09Rik,S,191,IPI00720110,Class I,0.999998,KVLTANSNPSSPS,-2.41,Negative,-2.74,...,195.38,Q14AR7;Q2NL54,IPI00720110,191,IPI00720110,RIKEN cDNA 1600027N09 gene,_VITANS(ph)NPSS(ph)PSAAK_,1600027N09Rik;mCG_6726;RP23-74K21.4;RP23-74K21...,"MCG6726, isoform CRA_f;Novel protein (Possible...",1.960000e-12
2,1700080G18Rik,S,308,IPI00108279,Class I,0.998163,FKAPTRSVNGELW,-2.64,Negative,-2.27,...,70.98,Q9CVQ8,IPI00108279;IPI00808211,308;382,IPI00108279;IPI00808211,hypothetical protein LOC73533;hypothetical pro...,_APTRS(ph)VNGEIWYKAIW_,1700080G18Rik,Putative uncharacterized protein,1.435810e-03
3,8030462N17Rik,S,143,IPI00466384,Class I,1.000000,GRKSRRSRSESET,3.41,Positive,2.73,...,256.52,Q8BH50-1;Q8BH50;Q0VAW6;Q19SD0;Q8BH50-2;Q19SC5,IPI00466384;IPI00874579;IPI00719980,143,IPI00466384,"RIKEN cDNA 8030462N17, isoform CRA_a;Isoform 2...",_S(ph)RS(ph)ES(ph)ETSTMAAK_,8030462N17Rik;mCG_18277;Akd2;8030462N17Rik;Akd2,Uncharacterized protein C18orf25 homolog;Putat...,6.170000e-55
4,8030462N17Rik,S,67,IPI00466384,Class I,0.999990,ELRRDSSESQLAS,-1.09,Negative,-1.00,...,257.50,Q8BH50-1;Q8BH50;Q0VAW6;Q19SD0;Q8BH50-2,IPI00466384;IPI00874579,67,IPI00466384,"RIKEN cDNA 8030462N17, isoform CRA_a;Isoform 2...",_RDS(ph)S(ph)ESQIASTESDKPTTGR_,8030462N17Rik;mCG_18277;Akd2,Uncharacterized protein C18orf25 homolog;Putat...,5.960000e-26


In [17]:
df = df[['Uniprot ID', 'Phosphorylated Amino Acid', 'Sequence Window', 'IPI 3.68 Position in Protein', 'Inhibitor Exps Insulin/Starved Median (log2)']].copy()
df['middle'] = df['Sequence Window'].apply(lambda x: x[6])
df['pass'] = df['middle'] == df['Phosphorylated Amino Acid']

#how many did not pass
print(df['pass'].value_counts())

#how many Uniprot IDs are blank
print(df['Uniprot ID'].isnull().sum())

#remove the empty Uniprot IDs and the ones that did not pass
df = df[(df['Uniprot ID'].notnull()) & df['pass']].copy()

df['ID List'] = df['Uniprot ID'].apply(lambda x: set([y.split('-')[0] for y in x.split(';')]))
data_exploded_df = df.explode('ID List').reset_index(drop=True)
data_exploded_df['motif'] = data_exploded_df['Sequence Window'].apply(lambda x: x[1:11])

display(data_exploded_df.head())

df = data_exploded_df[['ID List', 'IPI 3.68 Position in Protein', 'motif', 'Inhibitor Exps Insulin/Starved Median (log2)']].copy()
#rename columns to Uniprot_ID, Position, Motif, and log2FC
df.columns = ['Uniprot_ID', 'Position', 'Motif', 'log2FC']

#remove duplicate rows
df = df.drop_duplicates()

display(df.head())


human_experiment_file = os.path.join(test_dir, 'human_experiment.csv')
print(f'Number of phosphosites: {len(df)}')
df.to_csv(human_experiment_file, index=False)

pass
True    1798
Name: count, dtype: int64
8


,Uniprot ID,Phosphorylated Amino Acid,Sequence Window,IPI 3.68 Position in Protein,Inhibitor Exps Insulin/Starved Median (log2),middle,pass,ID List,motif
0,Q3UGE0;Q3UNN1;Q5DTS3;Q8C2A3;O55131,T,QQNSSRTLEKNKK,426,-1.41,T,True,Q5DTS3,QNSSRTLEKN
1,Q3UGE0;Q3UNN1;Q5DTS3;Q8C2A3;O55131,T,QQNSSRTLEKNKK,426,-1.41,T,True,Q3UNN1,QNSSRTLEKN
2,Q3UGE0;Q3UNN1;Q5DTS3;Q8C2A3;O55131,T,QQNSSRTLEKNKK,426,-1.41,T,True,O55131,QNSSRTLEKN
3,Q3UGE0;Q3UNN1;Q5DTS3;Q8C2A3;O55131,T,QQNSSRTLEKNKK,426,-1.41,T,True,Q8C2A3,QNSSRTLEKN
4,Q3UGE0;Q3UNN1;Q5DTS3;Q8C2A3;O55131,T,QQNSSRTLEKNKK,426,-1.41,T,True,Q3UGE0,QNSSRTLEKN


,Uniprot_ID,Position,Motif,log2FC
0,Q5DTS3,426,QNSSRTLEKN,-1.41
1,Q3UNN1,426,QNSSRTLEKN,-1.41
2,O55131,426,QNSSRTLEKN,-1.41
3,Q8C2A3,426,QNSSRTLEKN,-1.41
4,Q3UGE0,426,QNSSRTLEKN,-1.41


Number of phosphosites: 5936


In [18]:
column_names_dict={'id':'Uniprot_ID', 'site':'Position', 'peptide':'Motif', 'log2fc':'log2FC', 'dependent': None}

df = pd.read_csv(human_experiment_file)

session = Session(0, 'human', df, column_names_dict, scoring, background, ortholog_manager, id_type='UniProtKB', ambiguous=True, debug=True)

3924    Q8BTY2_238
Name: __ENTRY_ID, dtype: object
Elapsed time for percentiles and matches: 17.48 seconds
